## Appendix A
The following snippets will focus on Appendix A.exercises

#### A.1-2

In [8]:
import torch

tensor0d = torch.tensor(1)
tensor1d = torch.tensor([1, 2, 3])
tensor2d = torch.tensor([[1, 2],
                        [3, 4]])
tensor3d = torch.tensor([[[1, 2], [3, 4]],
                        [[5, 6], [7, 8]]])

print(tensor1d.dtype)

torch.int64


In [9]:
floatvec = torch.tensor([1.0, 2.0, 3.0])
print(floatvec.dtype)

torch.float32


In [10]:
floatvec = tensor1d.to(torch.float32)
print(floatvec.dtype) 

torch.float32


In [11]:
tensor2d = torch.tensor([[1, 2, 3],
                        [4, 5, 6]])
print(tensor2d)

print(tensor2d.shape)

tensor([[1, 2, 3],
        [4, 5, 6]])
torch.Size([2, 3])


In [12]:
print(tensor2d.reshape(3,2))

print(tensor2d.view(3,2))

tensor([[1, 2],
        [3, 4],
        [5, 6]])
tensor([[1, 2],
        [3, 4],
        [5, 6]])


In [13]:
print(tensor2d.T)

tensor([[1, 4],
        [2, 5],
        [3, 6]])


In [14]:
print(tensor2d.matmul(tensor2d.T))

tensor([[14, 32],
        [32, 77]])


#### A.3-4

In [15]:
import torch
import torch.nn.functional as F

y = torch.tensor([1.0]) # true label
x1 = torch.tensor([1.1]) # input feature
w1 = torch.tensor([2.2]) # weight parameter
b = torch.tensor([0.0]) # bias unit
z = x1 * w1 + b # net input
a = torch.sigmoid(z) #activation & output
loss = F.binary_cross_entropy(a, y) 

In [16]:
from torch.autograd import grad

y = torch.tensor([1.0])
x1 = torch.tensor([1.1])
w1 = torch.tensor([2.2], requires_grad=True)
b = torch.tensor([0.0], requires_grad=True)

z = x1 * w1 + b
a = torch.sigmoid(z)

loss = F.binary_cross_entropy(a, y)

grad_L_w1 = grad(loss, w1, retain_graph=True) # retain_graph=True keeps the computation graph in memory
grad_L_b = grad(loss, b, retain_graph=True)

# the resulting values of the loss gradiants given the model's parameters follow:

print(grad_L_w1)
print(grad_L_b)

(tensor([-0.0898]),)
(tensor([-0.0817]),)


In [17]:
loss.backward()
print(w1.grad)
print(b.grad)

tensor([-0.0898])
tensor([-0.0817])


#### A.5 - Implementing Multilayer Neural Networks

In [18]:
class NeuralNetwork(torch.nn.Module):
    def __init__(self, num_inputs, num_outputs):
        super().__init__()   
# coding the number of inputs and outputs as vairables allows us to reuse the same 
# code for datasets with different numbers of features and classes

        self.layers = torch.nn.Sequential(

            # 1st hidden layer
            torch.nn.Linear(num_inputs, 30), # The Linear layer takes teh number of input and output nodes as args
            torch.nn.ReLU(), # Nonlinear activation functions are plaed between hidden layers

            # 2nd hidden layer
            torch.nn.Linear(30, 20), # The number of output nodes of one hidden layer has to match the number of input nodes of the next layer
            torch.nn.ReLU(),

            # output layer
            torch.nn.Linear(20, num_outputs)

        )

    def forward(self, x):
        logits = self.layers(x)
        return logits # the outputs of the last layer are called logits
    
# Instantiate a new neural network

model = NeuralNetwork(50, 3)

print(model)

NeuralNetwork(
  (layers): Sequential(
    (0): Linear(in_features=50, out_features=30, bias=True)
    (1): ReLU()
    (2): Linear(in_features=30, out_features=20, bias=True)
    (3): ReLU()
    (4): Linear(in_features=20, out_features=3, bias=True)
  )
)


In [19]:
num_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print("Total number of trainable model parameters:", num_params)

Total number of trainable model parameters: 2213


In [20]:
print(model.layers[0].weight)

Parameter containing:
tensor([[-0.1225,  0.0447, -0.1348,  ..., -0.1390,  0.0863, -0.1192],
        [ 0.0366, -0.0590,  0.0906,  ..., -0.0360,  0.1010, -0.1341],
        [ 0.0741,  0.0876,  0.0331,  ..., -0.1305,  0.1180, -0.1076],
        ...,
        [-0.1327,  0.0675, -0.0327,  ...,  0.0852, -0.1070, -0.0542],
        [ 0.1305, -0.0717, -0.0972,  ..., -0.0855, -0.1274,  0.1361],
        [ 0.0763,  0.0593,  0.1116,  ...,  0.1412,  0.1362,  0.0679]],
       requires_grad=True)


In [21]:
print(model.layers[0].weight.shape)

torch.Size([30, 50])


In [22]:
torch.manual_seed(123)
model = NeuralNetwork(50, 3)
print(model.layers[0].weight)

Parameter containing:
tensor([[-0.0577,  0.0047, -0.0702,  ...,  0.0222,  0.1260,  0.0865],
        [ 0.0502,  0.0307,  0.0333,  ...,  0.0951,  0.1134, -0.0297],
        [ 0.1077, -0.1108,  0.0122,  ...,  0.0108, -0.1049, -0.1063],
        ...,
        [-0.0787,  0.1259,  0.0803,  ...,  0.1218,  0.1303, -0.1351],
        [ 0.1359,  0.0175, -0.0673,  ...,  0.0674,  0.0676,  0.1058],
        [ 0.0790,  0.1343, -0.0293,  ...,  0.0344, -0.0971, -0.0509]],
       requires_grad=True)


In [23]:
torch.manual_seed(123)
X = torch.rand((1, 50))
out = model(X)
print(out)

tensor([[-0.1262,  0.1080, -0.1792]], grad_fn=<AddmmBackward0>)


In [24]:
# drop construction of the computational graph for backpropagation to save memory/computation
with torch.no_grad():
    out = model(X)
print(out)

tensor([[-0.1262,  0.1080, -0.1792]])


In [25]:
with torch.no_grad():
    out = torch.softmax(model(X), dim=1)
print(out)

tensor([[0.3113, 0.3934, 0.2952]])


#### A.6 - Setting Up Efficient Data Loaders

In [26]:
X_train = torch.tensor([
    [-1.2, 3.1],
    [-0.9, 2.9],
    [-0.5, 2.6],
    [2.3, -1.1],
    [2.7, -1.5],
])
y_train = torch.tensor([0, 0, 0 ,1, 1])

X_test = torch.tensor([
    [-0.8, 2.8],
    [2.6, -1.6],
])
y_test = torch.tensor([0, 1])

In [27]:
from torch.utils.data import Dataset

class ToyDataset(Dataset):
    def __init__(self, X, y):
        self.features = X
        self.labels = y
# instructions for retrieving exactly one data record and the corresponding label
    def __getitem__(self, index):
        one_x = self.features[index]
        one_y = self.labels[index]
        return one_x, one_y
# instructions for returning the total length of the dataset
    def __len__(self):
        return self.labels.shape[0]
    
train_ds = ToyDataset(X_train, y_train)
test_ds = ToyDataset(X_test, y_test)

In [28]:
# check the length of the training dataset
print(len(train_ds)) 

5


In [29]:
from torch.utils.data import DataLoader

torch.manual_seed(123)

# The ToyDataset instance created earlier serves as input to the data loader
train_loader = DataLoader(
        dataset=train_ds,
        batch_size=2,
        shuffle=True, # whether or not to shuffle the data
        num_workers=0 # number of background processes
)

test_loader = DataLoader(
    dataset=test_ds,
    batch_size=2,
    shuffle=False, # it isn't nececessary to shuffle test data
    num_workers=0
)


In [30]:
for idx, (x, y) in enumerate(train_loader):
    print(f"Batch {idx+1}:", x, y)


Batch 1: tensor([[ 2.3000, -1.1000],
        [-0.9000,  2.9000]]) tensor([1, 0])
Batch 2: tensor([[-1.2000,  3.1000],
        [-0.5000,  2.6000]]) tensor([0, 0])
Batch 3: tensor([[ 2.7000, -1.5000]]) tensor([1])


For what it's worth, the above flips Batch 1 and 2 from what is shown in the book, perhaps because I ran the prior block already but I'm not sure.  I do know at least that each run will shuffle the order of the examples so it may just be related.  

In [31]:
# drop last batch due to 5/2 not being clean and teh smaller last batch potentially disturbing convergence

train_loader = DataLoader(
    dataset=train_ds,
    batch_size=2,
    shuffle=True,
    num_workers=0,
    drop_last=True
)

for idx, (x, y) in enumerate(train_loader):
    print(f"Batch {idx+1}:", x, y)

Batch 1: tensor([[-1.2000,  3.1000],
        [-0.5000,  2.6000]]) tensor([0, 0])
Batch 2: tensor([[ 2.3000, -1.1000],
        [-0.9000,  2.9000]]) tensor([1, 0])


#### A.7 - A Typical Training Loop

In [32]:
import torch.nn.functional as F
import torch

torch.manual_seed(123)
model = NeuralNetwork(num_inputs=2, num_outputs=2) # dataset has two features and two classes
optimizer = torch.optim.SGD(
    model.parameters(), lr=0.5 # optimizer needs to know which parameters to optimize
)

num_epochs = 3
for epoch in range(num_epochs):

    model.train()

    for batch_idx, (features, labels) in enumerate(train_loader):
        logits = model(features)

        loss = F.cross_entropy(logits, labels)

        optimizer.zero_grad() # sets teh gradients from the previous round to 0 to prevent unintended gradient accumulation
        loss.backward() # computes the gradients of the loss given the model parameters
        optimizer.step() # updates the model parameters based on the computed gradients

        ### LOGGING
        print(f"Epoch: {epoch+1:03d}/{num_epochs:03d}"
              f" | Batch {batch_idx:03d}/{len(train_loader):03d}"
              f" | Train Loss: {loss:.2f}")
        
    model.eval() #insert optional model evaluation code

Epoch: 001/003 | Batch 000/002 | Train Loss: 0.75
Epoch: 001/003 | Batch 001/002 | Train Loss: 0.65
Epoch: 002/003 | Batch 000/002 | Train Loss: 0.44
Epoch: 002/003 | Batch 001/002 | Train Loss: 0.13
Epoch: 003/003 | Batch 000/002 | Train Loss: 0.03
Epoch: 003/003 | Batch 001/002 | Train Loss: 0.00


In [35]:
num_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print("Total number of trainable model parameters:", num_params)

Total number of trainable model parameters: 752


In [36]:
# predictions time
model.eval()
with torch.no_grad():
    outputs = model(X_train)
print(outputs)

tensor([[ 2.8569, -4.1618],
        [ 2.5382, -3.7548],
        [ 2.0944, -3.1820],
        [-1.4814,  1.4816],
        [-1.7176,  1.7342]])


In [37]:
# to obatin class membership probabilities:

torch.set_printoptions(sci_mode=False)
probas = torch.softmax(outputs, dim=1)
print(probas)

tensor([[    0.9991,     0.0009],
        [    0.9982,     0.0018],
        [    0.9949,     0.0051],
        [    0.0491,     0.9509],
        [    0.0307,     0.9693]])


In [38]:
predictions = torch.argmax(probas, dim=1)
print(predictions)

tensor([0, 0, 0, 1, 1])


In [39]:
predictions = torch.argmax(outputs, dim=1)
print(predictions)

tensor([0, 0, 0, 1, 1])


In [40]:
predictions == y_train

tensor([True, True, True, True, True])

In [41]:
torch.sum(predictions == y_train)

tensor(5)

In [42]:
def compute_accuracy(model, dataloader):

    model = model.eval()
    correct = 0.0
    total_examples = 0

    for idx, (features, labels) in enumerate(dataloader):

        with torch.no_grad():
            logits = model(features)

        predictions = torch.argmax(logits, dim=1)
        compare = labels == predictions # returns a tensor of true/false values depending on whether labels match
        correct += torch.sum(compare) # sum operation counts number of True values
        total_examples += len(compare)

    return (correct / total_examples).item() # fraction of correct prediction, a value between 0 and 1. .item() returns the value of the tensor as a Python float

In [43]:
print(compute_accuracy(model, train_loader))

1.0


In [44]:
print(compute_accuracy(model, test_loader))

1.0


#### A.8 Saving and Loading Models

In [45]:
torch.save(model.state_dict(), "model.pth")

In [46]:
model = NeuralNetwork(2, 2)
model.load_state_dict(torch.load("model.pth"))

<All keys matched successfully>

#### A.9 Optimizing Training Performance with GPUs

In [47]:
print(torch.cuda.is_available())

True


In [48]:
tensor_1 = torch.tensor([1., 2., 3.])
tensor_2 = torch.tensor([4., 5., 6.])
print(tensor_1 + tensor_2)

tensor([5., 7., 9.])


In [49]:
tensor_1 = tensor_1.to("cuda")
tensor_2 = tensor_2.to("cuda")
print(tensor_1 + tensor_2)

tensor([5., 7., 9.], device='cuda:0')


In [51]:
torch.manual_seed(123)
model = NeuralNetwork(num_inputs=2, num_outputs=2)

device = torch.device("cuda")
model = model.to(device)

optimizer = torch.optim.SGD(model.parameters(), lr=0.5)

num_epochs = 3

for epoch in range(num_epochs):

    model.train()
    for batch_idx, (features, labels) in enumerate(train_loader):
        features, labels = features.to(device), labels.to(device)
        logits = model(features)
        loss = F.cross_entropy(logits, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        ### LOGGING
        print(f"Epoch: {epoch+1:03d}/{num_epochs:03d}"
              f" | Batch {batch_idx:03d}/{len(train_loader):03d}"
              f" | Train Loss: {loss:.2f}")
    
    model.eval()

Epoch: 001/003 | Batch 000/002 | Train Loss: 0.75
Epoch: 001/003 | Batch 001/002 | Train Loss: 0.65
Epoch: 002/003 | Batch 000/002 | Train Loss: 0.44
Epoch: 002/003 | Batch 001/002 | Train Loss: 0.13
Epoch: 003/003 | Batch 000/002 | Train Loss: 0.03
Epoch: 003/003 | Batch 001/002 | Train Loss: 0.00


In [52]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")